# BAX pore assembly
Initial condition:
\begin{equation*}
P_0(x)=\gamma \cdot \exp\left(-\frac{1}{2} (x-\mu)^T C^{-1} (x-\mu)\right),
\end{equation*}
with $C=0.2$, $\mu=(80,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,100,\,0)$ and $\gamma$ was determined by the condition $\sum_{x\in\Omega^{\zeta,\eta}} P_0(x)=1$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pysb_model_bax as pysb_model
from pysb.simulator import StochKitSimulator

from ssa_helper import *

## Simulation parameters

In [2]:
n_time = 10 + 1
tspan = np.linspace(0.0, 20000.0, n_time)

# This defines the small sample space
liml = np.array([0, 0, 0, 0, 98, 0, 78, 0, 0, 0,0])
n = np.array([4, 4, 4, 4, 5, 4, 5, 4, 4, 4, 4])

sweeps = 1e4

m = n.size # len(pysb_model.model.observables)

## Set up the initial condition

In [3]:
C = 0.2
Cinv = 1 / C
mu = np.array([0, 0, 0, 0, 100, 0, 80, 0, 0, 0, 0])

def eval_P0(gamma: float, x: np.ndarray, mu: np.ndarray) -> float:
    return gamma * np.exp(-0.5 * Cinv * np.dot(np.transpose(x - mu), (x - mu)))

In [4]:
# Calculate the normalization
sum_vec = np.zeros(n.size)
for i, n_el in enumerate(n):
    for x in range(n_el):
        sum_vec[i] += eval_P0(1.0, x + liml[i], mu[i])
gamma = 1 / np.prod(sum_vec)

## Run StochKit simulation

In [5]:
result = runSSAWithP0(lambda x: eval_P0(gamma, x, mu), sweeps, tspan, n, liml, pysb_model.model)
np.save("scripts/reference_solutions/bax_ssa_general_ref_1e4.npy", result)